In [0]:
import pymysql.cursors
import pandas as pd
import pymysql
import pyspark.sql.functions as F

In [0]:
host = "db4free.net"
db = "exocube"
uname = "sample_jenish"
pas = "Jenu7492@patel"

In [0]:
Source_Table_Name = "EMP_INFO"

In [0]:
def get_connection(host, uname, pas, db):
    try:
        sql_connection = pymysql.connect(host=host,user=uname,passwd=pas,db=db, cursorclass=pymysql.cursors.DictCursor, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymysql.OperationalError as e:
        if e.args[0] == 2003:
            print('server not found')
        elif e.args[0] == 1044:
            print('database not found')
        elif e.args[0] == 1045:
            print('username or password incorrect')
    except:
        print("Unknown error occurred")   
    else:
        return sql_connection

In [0]:
def display_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        sql_exec = cursor.execute(sql_disqry)
        if sql_exec:
            print(sql_exec)
            sql_data = pd.DataFrame(cursor.fetchall())
            result = sql_data.head(n=None)
            #display(sql_data.loc[1])
            display(result)
            spark_df = spark.createDataFrame(result)
            df = spark_df.toPandas()
            #spark_df.orderBy("Emp_Id").write.mode("overwrite").saveAsTable(Source_Table_Name)
            
            pds = spark.sql("(select * FROM {})".format(Source_Table_Name))
            pdf = pds.withColumn("Hash_Key",F.sha2(F.concat('Emp_Id','Emp_name', 'Designation', 'Department', 'City', 'Salary', F.unix_timestamp('Joining_Date')),256))
            #pdf = pds.withColumn("Hash_Key",F.md5(F.concat('Emp_Id', F.trim(F.lower('Emp_name')), F.trim(F.lower('Designation')), F.trim(F.lower('Department')), F.trim(F.lower('City')), 'Salary', F.unix_timestamp('Joining_Date'))))
            pnd = pdf.toPandas()
            pd_df = pnd[['Emp_Id', 'Hash_Key']]
            #df_all = df.compare(pd_df)
            #display(df_all)
        else:
            print(sql_exec)
            print("No Record")
    except pymysql.ProgrammingError as e:
        if e.args[0] == 1146:
            print('table doesnt exist')
        elif e.args[0] == 1064:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_connection(host, uname, pas, db)

In [0]:
sql_disqry ="(select Emp_Id, sha2(CONCAT(Emp_Id, Emp_name, Designation, Department, City, Salary, UNIX_TIMESTAMP(Joining_Date)),256)as Hash_Key FROM {})".format(Source_Table_Name)
display_data(sql_disqry, sql_conn)

16


Emp_Id,Hash_Key
1,8bd5ba1515eb85223285f93239c1fc707e046c04720fab1458b981f6651657a5
4,dfe448dc6fb5bdb1aeff670d6d0bbd564418613beb725841e803eb0a5ecb2833
6,440a2ad1a4472c9703dfeac22b8a6465004710701b5ece5b25b53aa8f8db7493
7,e1999a3a4b0815464685bdcf1e80bd9635ecf8e1394b3d8d2f692cd4a9024234
8,b5cd98edfd177e0cbf476728375c679147ba61a95cd62a89e919d7cb309525c1
9,bebd1156519378d61fced177c271841cfd30d3ef4af624a81c3e37c9344f42ef
11,12b200ab4af6e9c866382a8e0f0d0770c76e06704030b1feaaa0090e5598a165
12,b6525fd429191431da1120e2e89ab4c426915fdeefacc28d0fc83472a96b7d22
13,4f16df96196a8a2eb75c579ef792b961c2c000e47070264c157a3cea7bcf86ed
14,4367e01f192da2f38fb5612007ccdefacb5477b010077542a396fc4bb0432ba8


In [0]:
pip install pymysql

Python interpreter will be restarted.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-71826fbb-3f5b-4a01-a65b-5777b4ea37fe/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
